# Time series analysis tutorial

데이터 링크는 [Superstore sales data](https://community.tableau.com/docs/DOC-1236)입니다.

튜토리얼의 목적은 **<font color=red>시계열 데이터를 분석하여 furniture(가구)의 판매를 예측</font>**하는 것입니다.

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import itertools
import matplotlib

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
import statsmodels.api as sm

## 데이터 로딩

`pip install xlrd`

In [8]:
pd.options.display.max_rows=None
pd.options.display.max_columns=None

In [9]:
df = pd.read_excel("./input/Superstore.xls")
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [10]:
df.shape

(9994, 21)

*Category*컬럼에서 값이 *Furniture*인 데이터를 불러옵니다.

In [11]:
furniture = df.loc[df.Category == 'Furniture']
furniture.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
5,6,CA-2014-115812,2014-06-09,2014-06-14,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,90032,West,FUR-FU-10001487,Furniture,Furnishings,Eldon Expressions Wood and Plastic Desk Access...,48.8600,7,0.00,14.1694
10,11,CA-2014-115812,2014-06-09,2014-06-14,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,90032,West,FUR-TA-10001539,Furniture,Tables,Chromcraft Rectangular Conference Tables,1706.1840,9,0.20,85.3092


데이터의 시점과 종점을 살펴봅니다.

In [13]:
print('시점 :', furniture['Order Date'].min())
print('종점 :', furniture['Order Date'].max())

시점 : 2014-01-06 00:00:00
종점 : 2017-12-30 00:00:00


총 4년 동안의 데이터 입니다.

해당 컬럼의 데이터 타입을 살펴봅니다

In [18]:
type(furniture['Order Date'].min())

pandas._libs.tslib.Timestamp

## Data Preprocessing
- 사용하지 않을 컬럼을 제거하겠습니다.
- Missing value를 체크합니다.
- 컬럼을 aggregation합니다.
- 시계열 인덱싱으로 변환합니다.

### 1. 사용하지 않을 컬럼 제거
- 사용하는 컬럼은 시간 정보가 있는 *Order Date*와, 그 날의 판매량인 *Sales* 입니다.

In [19]:
columns = ['Row ID', 'Order ID', 'Ship Date', 'Ship Mode', 
           'Customer ID', 'Customer Name', 'Segment', 'Country', 
           'City', 'State', 'Postal Code', 'Region', 
           'Product ID', 'Category', 'Sub-Category', 'Product Name', 
           'Quantity', 'Discount', 'Profit']

In [21]:
furniture.drop(columns, axis=1, inplace=True)
furniture = furniture.sort_values('Order Date')

In [22]:
furniture.head()

,Order Date,Sales
7474,2014-01-06,2573.820
7660,2014-01-07,76.728
866,2014-01-10,51.940
716,2014-01-11,9.940
2978,2014-01-13,545.940


### 2. Missing value를 체크

In [23]:
furniture.isnull().sum()

Order Date    0
Sales         0
dtype: int64

### 3. 컬럼 aggregation

각 날짜별로 record의 수를 체크해봅니다.

In [29]:
furniture.groupby('Order Date').count().sort_values(by='Sales', ascending=False).head()

,Sales
Order Date,
2016-09-05,10
2016-12-25,9
2016-12-01,9
2017-11-19,9
2017-10-30,9


날짜별로 record의 수가 여러개 존재하는 경우가 있으므로, sales을 aggregation 합니다.

In [33]:
furniture = furniture.groupby('Order Date').agg({'Sales': 'sum'}).reset_index()
furniture.head()

,Order Date,Sales
0,2014-01-06,2573.820
1,2014-01-07,76.728
2,2014-01-10,51.940
3,2014-01-11,9.940
4,2014-01-13,879.939


### 4. Indexing

In [34]:
furniture = furniture.set_index('Order Date')
furniture.index

DatetimeIndex(['2014-01-06', '2014-01-07', '2014-01-10', '2014-01-11',
               '2014-01-13', '2014-01-14', '2014-01-16', '2014-01-19',
               '2014-01-20', '2014-01-21',
               ...
               '2017-12-18', '2017-12-19', '2017-12-21', '2017-12-22',
               '2017-12-23', '2017-12-24', '2017-12-25', '2017-12-28',
               '2017-12-29', '2017-12-30'],
              dtype='datetime64[ns]', name='Order Date', length=889, freq=None)

시계열 데이터는 다루기 힘든 경우가 많습니다. 미시적(micro)인 관점에서 살펴보아도 좋지만, 먼저 거시적(macro)인 단위에서의 경향성을 예측해보고, 범위를 좁혀가는 방법이 좋습니다.

따라서, 1일 판매량 단위로 끊어서보지 않고, 1달 판매량의 평균으로 살펴보도록 하겠습니다.

Pandas는 시계열 데이터를 다룰때, range를 다루기 쉬운 메소드를 제공해줍니다.

`resample` reference : https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.resample.html

In [45]:
x = furniture['Sales'].resample('MS')
x.head()

Order Date
2014-01-01    480.194231
2014-02-01    367.931600
2014-03-01    857.291529
2014-04-01    567.488357
2014-05-01    432.049188
Freq: MS, Name: Sales, dtype: float64